In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers

import pickle
import numpy as np

f2 = open('data/split_0.80_0.10_0.10_aug_to_2500_down_sample_True_instance0-9/val_data', 'rb')
X_sequence_val, X_feature_val, X_image_val, Y_val = pickle.load(f2)
f2.close()

from keras.preprocessing.sequence import pad_sequences
from util import preprocess

total_X_sequence_train = []
total_X_feature_train = []
# total_X_image_train = []
total_Y_train = []
for i in range(0,10):
    f = open('data/split_0.80_0.10_0.10_aug_to_2500_down_sample_True_instance0-9/train_data%d'%i, 'rb')
    X_sequence_train, X_feature_train, X_image_train, Y_train = pickle.load(f)
    X_sequence_train_paded = pad_sequences(X_sequence_train, dtype='float', padding='post', value = np.nan)
    X_sequence_train_trimed = preprocess(X_sequence_train_paded)
    X_feature_train = np.array(X_feature_train)
    Y_train = np.array(Y_train)-1

    Y_train = keras.utils.to_categorical(Y_train, num_classes=11)

    total_X_sequence_train.append(X_sequence_train_trimed)
    total_X_feature_train.append(X_feature_train)
    total_Y_train.append(Y_train)
    f.close()

X_sequence_val_paded = pad_sequences(X_sequence_val, dtype='float', padding='post', value = np.nan)
X_sequence_val_trimed = preprocess(X_sequence_val_paded)
X_feature_val = np.array(X_feature_val)
Y_val = np.array(Y_val)-1


Y_val = keras.utils.to_categorical(Y_val, num_classes=11)

In [3]:
from mix_models import rnn_with_feature

X_val = {'sequence':X_sequence_val_trimed, 'feature':X_feature_val}
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=860,
    decay_rate=0.85)

for i in range(1,10):
    model = rnn_with_feature(sequence_shape=(None, 3),feature_shape=(2,), num_classes=11)
    model.compile(
    optimizer=keras.optimizers.Adam(lr_schedule),
    # loss="sparse_categorical_crossentropy",
    loss=keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    # metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")],
    metrics=[keras.metrics.CategoricalAccuracy(name="acc")],
    run_eagerly=False,
    )
    callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath='models/bagging/rnn_with_feature%d/model_{epoch}'%i,
        save_freq='epoch'),
    keras.callbacks.TensorBoard(log_dir='models/bagging/rnn_with_feature%d/logs'%i)
    ]
    X_train = {'sequence':total_X_sequence_train[i], 'feature':total_X_feature_train[i]}
    model.fit(X_train, total_Y_train[i], batch_size=32, epochs=10, 
        validation_data=(X_val, Y_val), callbacks=callbacks)


2022-10-19 19:49:45.853449: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-19 19:49:46.743268: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22245 MB memory:  -> device: 0, name: GeForce RTX 3090, pci bus id: 0000:73:00.0, compute capability: 8.6
2022-10-19 19:49:46.744284: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22300 MB memory:  -> device: 1, name: GeForce RTX 3090, pci bus id: 0000:d5:00.0, compute capability: 8.6
2022-10-19 19:49:48.710931: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-

Epoch 1/10


2022-10-19 19:49:56.217074: I tensorflow/stream_executor/cuda/cuda_blas.cc:1760] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2022-10-19 19:49:56.385635: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8401
Could not load symbol cublasGetSmCountTarget from libcublas.so.11. Error: /home/zhkang/softwares/anaconda3/envs/DL/lib/python3.9/site-packages/tensorflow/python/../../../../libcublas.so.11: undefined symbol: cublasGetSmCountTarget


  1/860 [..............................] - ETA: 1:55:59 - loss: 4.9954 - acc: 0.0938

2022-10-19 19:49:57.639285: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-10-19 19:49:57.639321: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.


...

INFO:tensorflow:Assets written to: models/bagging/rnn_with_feature9/model_9/assets


Epoch 10/10
860/860 [==============================] - 48s 55ms/step - loss: 0.7967 - acc: 0.9110 - val_loss: 0.9921 - val_acc: 0.8199


INFO:tensorflow:Assets written to: models/bagging/rnn_with_feature9/model_10/assets


INFO:tensorflow:Assets written to: models/bagging/rnn_with_feature9/model_10/assets


: 